In [75]:
import numpy
import pandas
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
import sys
import numpy
import os
import scipy.stats
import math
import ast
import statsmodels.api as sm

In [76]:
sys.path.append("../continuous")
sys.path.append("../continuous/measurements")
sys.path.append("../utilities")
import ad_marsaglia as ad
from measurements import MEASUREMENTS_CONTINUOUS

In [77]:
mpl.style.use("ggplot")

In [78]:
from distributions.alpha import ALPHA
from distributions.arcsine import ARCSINE
from distributions.argus import ARGUS
from distributions.beta import BETA
from distributions.beta_prime import BETA_PRIME
from distributions.beta_prime_4p import BETA_PRIME_4P
from distributions.bradford import BRADFORD
from distributions.burr import BURR
from distributions.burr_4p import BURR_4P
from distributions.cauchy import CAUCHY
from distributions.chi_square import CHI_SQUARE
from distributions.chi_square_3p import CHI_SQUARE_3P
from distributions.dagum import DAGUM
from distributions.dagum_4p import DAGUM_4P
from distributions.erlang import ERLANG
from distributions.erlang_3p import ERLANG_3P
from distributions.error_function import ERROR_FUNCTION
from distributions.exponential import EXPONENTIAL
from distributions.exponential_2p import EXPONENTIAL_2P
from distributions.f import F
from distributions.fatigue_life import FATIGUE_LIFE
from distributions.folded_normal import FOLDED_NORMAL
from distributions.frechet import FRECHET
from distributions.f_4p import F_4P
from distributions.gamma import GAMMA
from distributions.gamma_3p import GAMMA_3P
from distributions.generalized_extreme_value import GENERALIZED_EXTREME_VALUE
from distributions.generalized_gamma import GENERALIZED_GAMMA
from distributions.generalized_gamma_4p import GENERALIZED_GAMMA_4P
from distributions.generalized_logistic import GENERALIZED_LOGISTIC
from distributions.generalized_normal import GENERALIZED_NORMAL
from distributions.generalized_pareto import GENERALIZED_PARETO
from distributions.gibrat import GIBRAT
from distributions.gumbel_left import GUMBEL_LEFT
from distributions.gumbel_right import GUMBEL_RIGHT
from distributions.half_normal import HALF_NORMAL
from distributions.hyperbolic_secant import HYPERBOLIC_SECANT
from distributions.inverse_gamma import INVERSE_GAMMA
from distributions.inverse_gamma_3p import INVERSE_GAMMA_3P
from distributions.inverse_gaussian import INVERSE_GAUSSIAN
from distributions.inverse_gaussian_3p import INVERSE_GAUSSIAN_3P
from distributions.johnson_sb import JOHNSON_SB
from distributions.johnson_su import JOHNSON_SU
from distributions.kumaraswamy import KUMARASWAMY
from distributions.laplace import LAPLACE
from distributions.levy import LEVY
from distributions.loggamma import LOGGAMMA
from distributions.logistic import LOGISTIC
from distributions.loglogistic import LOGLOGISTIC
from distributions.loglogistic_3p import LOGLOGISTIC_3P
from distributions.lognormal import LOGNORMAL
from distributions.maxwell import MAXWELL
from distributions.moyal import MOYAL
from distributions.nakagami import NAKAGAMI
from distributions.nc_chi_square import NC_CHI_SQUARE
from distributions.nc_f import NC_F
from distributions.nc_t_student import NC_T_STUDENT
from distributions.normal import NORMAL
from distributions.pareto_first_kind import PARETO_FIRST_KIND
from distributions.pareto_second_kind import PARETO_SECOND_KIND
from distributions.pert import PERT
from distributions.power_function import POWER_FUNCTION
from distributions.rayleigh import RAYLEIGH
from distributions.reciprocal import RECIPROCAL
from distributions.rice import RICE
from distributions.semicircular import SEMICIRCULAR
from distributions.trapezoidal import TRAPEZOIDAL
from distributions.triangular import TRIANGULAR
from distributions.t_student import T_STUDENT
from distributions.t_student_3p import T_STUDENT_3P
from distributions.uniform import UNIFORM
from distributions.weibull import WEIBULL
from distributions.weibull_3p import WEIBULL_3P


In [79]:
_all_distributions = [
    ALPHA,
    ARCSINE,
    ARGUS,
    BETA,
    BETA_PRIME,
    BETA_PRIME_4P,
    BRADFORD,
    BURR,
    BURR_4P,
    CAUCHY,
    CHI_SQUARE,
    CHI_SQUARE_3P,
    DAGUM,
    DAGUM_4P,
    ERLANG,
    ERLANG_3P,
    ERROR_FUNCTION,
    EXPONENTIAL,
    EXPONENTIAL_2P,
    F,
    FATIGUE_LIFE,
    FOLDED_NORMAL,
    FRECHET,
    F_4P,
    GAMMA,
    GAMMA_3P,
    GENERALIZED_EXTREME_VALUE,
    GENERALIZED_GAMMA,
    GENERALIZED_GAMMA_4P,
    GENERALIZED_LOGISTIC,
    GENERALIZED_NORMAL,
    GENERALIZED_PARETO,
    GIBRAT,
    GUMBEL_LEFT,
    GUMBEL_RIGHT,
    HALF_NORMAL,
    HYPERBOLIC_SECANT,
    INVERSE_GAMMA,
    INVERSE_GAMMA_3P,
    INVERSE_GAUSSIAN,
    INVERSE_GAUSSIAN_3P,
    JOHNSON_SB,
    JOHNSON_SU,
    KUMARASWAMY,
    LAPLACE,
    LEVY,
    LOGGAMMA,
    LOGISTIC,
    LOGLOGISTIC,
    LOGLOGISTIC_3P,
    LOGNORMAL,
    MAXWELL,
    MOYAL,
    NAKAGAMI,
    NC_CHI_SQUARE,
    NC_F,
    NC_T_STUDENT,
    NORMAL,
    PARETO_FIRST_KIND,
    PARETO_SECOND_KIND,
    PERT,
    POWER_FUNCTION,
    RAYLEIGH,
    RECIPROCAL,
    RICE,
    SEMICIRCULAR,
    TRAPEZOIDAL,
    TRIANGULAR,
    T_STUDENT,
    T_STUDENT_3P,
    UNIFORM,
    WEIBULL,
    WEIBULL_3P,
]

In [80]:
def test_chi_square_continuous(data, distribution, measurements):
    ## Parameters and preparations
    N = measurements.length
    num_bins = measurements.num_bins
    frequencies, bin_edges = numpy.histogram(data, num_bins)
    freedom_degrees = num_bins - 1 - distribution.get_num_parameters()

    ## Calculation of errors
    errors = []
    for i, observed in enumerate(frequencies):
        lower = bin_edges[i]
        upper = bin_edges[i + 1]
        expected = N * (distribution.cdf(upper) - distribution.cdf(lower))
        errors.append(((observed - expected) ** 2) / expected)

    ## Calculation of indicators
    statistic_chi2 = sum(errors)
    critical_value = scipy.stats.chi2.ppf(0.95, freedom_degrees)
    p_value = 1 - scipy.stats.chi2.cdf(statistic_chi2, freedom_degrees)
    rejected = statistic_chi2 >= critical_value

    ## Construction of answer
    result_test_chi2 = {"test_statistic": statistic_chi2, "critical_value": critical_value, "p-value": p_value, "rejected": rejected}

    return result_test_chi2

In [81]:
def test_kolmogorov_smirnov_continuous(data, distribution, measurements):
    ## Parameters and preparations
    N = measurements.length
    data.sort()
    
    ## Calculation of errors
    errors = []
    for i in range(N):
        Sn = (i + 1) / N
        if i < N - 1:
            if (data[i] != data[i + 1]):
                Fn = distribution.cdf(data[i])
                errors.append(abs(Sn - Fn))
            else:
                Fn = 0
        else:
            Fn = distribution.cdf(data[i])
            errors.append(abs(Sn - Fn))
    
    ## Calculation of indicators
    statistic_ks = max(errors)
    critical_value = scipy.stats.kstwo.ppf(0.95, N)
    p_value = 1 -  scipy.stats.kstwo.cdf(statistic_ks, N)
    rejected = statistic_ks >= critical_value
    
    ## Construction of answer
    result_test_ks = {
        "test_statistic": statistic_ks, 
        "critical_value": critical_value, 
        "p-value": p_value,
        "rejected": rejected
    }
    
    return result_test_ks

In [82]:
def test_anderson_darling_continuous(data, distribution, measurements):
    ## Parameters and preparations
    N = measurements.length
    data.sort()
    
    ## Calculation S
    S = 0
    for k in range(N):
        c1 = math.log(distribution.cdf(data[k]))
        c2 = math.log(1 - distribution.cdf(data[N - k - 1]))
        c3 = (2 * (k + 1) - 1) / N
        S += c3 * (c1 + c2)
    
    ## Calculation of indicators
    A2 = -N - S
    critical_value = ad.ad_critical_value(0.95, N)
    p_value = ad.ad_p_value(N, A2)
    rejected = A2 >= critical_value
    
    ## Construction of answer
    result_test_ad = {
        "test_statistic": A2, 
        "critical_value": critical_value,
        "p-value": p_value,
        "rejected": rejected
    }
    
    return result_test_ad

In [83]:
path = "./data_1000/data_uniform.txt"
sample_distribution_file = open(path, "r")
data = [float(x.replace(",", ".")) for x in sample_distribution_file.read().splitlines()]

In [84]:
data = pandas.Series(sm.datasets.elnino.load_pandas().data.set_index("YEAR").values.ravel())
data

0      23.11
1      24.20
2      25.37
3      23.86
4      23.03
       ...  
727    19.49
728    19.28
729    19.73
730    20.44
731    22.07
Length: 732, dtype: float64

In [85]:
measurements = MEASUREMENTS_CONTINUOUS(data)

In [86]:
## Calculae Histogram
num_bins = measurements.num_bins_doane()
frequencies, bin_edges = numpy.histogram(data, num_bins, density=True)
central_values = [(bin_edges[i] + bin_edges[i + 1]) / 2 for i in range(len(bin_edges) - 1)]


In [87]:
columns = pandas.MultiIndex.from_product([["chi_square", "kolmogorov_smirnov", "anderson_darling"], ["test_statistic", "critical_value", "p_value", "rejected"]])
df = pandas.DataFrame(columns=columns)
df


Empty DataFrame
Columns: [(chi_square, test_statistic), (chi_square, critical_value), (chi_square, p_value), (chi_square, rejected), (kolmogorov_smirnov, test_statistic), (kolmogorov_smirnov, critical_value), (kolmogorov_smirnov, p_value), (kolmogorov_smirnov, rejected), (anderson_darling, test_statistic), (anderson_darling, critical_value), (anderson_darling, p_value), (anderson_darling, rejected)]
Index: []

In [88]:
for distribution_class in _all_distributions:
    distribution_name = distribution_class.__name__.lower()
    print(distribution_name)

    validate_estimation = True
    sse = 0
    try:
        distribution = distribution_class(measurements)
        pdf_values = [distribution.pdf(c) for c in central_values]
        sse = numpy.sum(numpy.power(frequencies - pdf_values, 2.0))
    except:
        validate_estimation = False
    print(sse)

    if validate_estimation and not math.isnan(sse):
        try:
            chi2_test = test_chi_square_continuous(data, distribution, measurements)
            if numpy.isnan(chi2_test["test_statistic"]) == False and math.isinf(chi2_test["test_statistic"]) == False and chi2_test["test_statistic"] > 0:
                df.loc[distribution_name, ("chi_square", "test_statistic")] = chi2_test["test_statistic"]
                df.loc[distribution_name, ("chi_square", "critical_value")] = chi2_test["critical_value"]
                df.loc[distribution_name, ("chi_square", "p_value")] = chi2_test["p-value"]
                df.loc[distribution_name, ("chi_square", "rejected")] = chi2_test["rejected"]
        except:
            pass

        try:
            ks_test = test_kolmogorov_smirnov_continuous(data, distribution, measurements)
            if numpy.isnan(ks_test["test_statistic"]) == False and math.isinf(ks_test["test_statistic"]) == False and ks_test["test_statistic"] > 0:
                df.loc[distribution_name, ("kolmogorov_smirnov", "test_statistic")] = ks_test["test_statistic"]
                df.loc[distribution_name, ("kolmogorov_smirnov", "critical_value")] = ks_test["critical_value"]
                df.loc[distribution_name, ("kolmogorov_smirnov", "p_value")] = ks_test["p-value"]
                df.loc[distribution_name, ("kolmogorov_smirnov", "rejected")] = ks_test["rejected"]
        except:
            pass

        try:
            ad_test = test_anderson_darling_continuous(data, distribution, measurements)
            if numpy.isnan(ad_test["test_statistic"]) == False and math.isinf(ad_test["test_statistic"]) == False and ad_test["test_statistic"] > 0:
                df.loc[distribution_name, ("anderson_darling", "test_statistic")] = ad_test["test_statistic"]
                df.loc[distribution_name, ("anderson_darling", "critical_value")] = ad_test["critical_value"]
                df.loc[distribution_name, ("anderson_darling", "p_value")] = ad_test["p-value"]
                df.loc[distribution_name, ("anderson_darling", "rejected")] = ad_test["rejected"]
        except:
            pass

        if distribution_name in df.index:
            df.loc[distribution_name, "sse"] = sse
            df.loc[distribution_name, "parameters"] = str(distribution.parameters)
            df.loc[distribution_name, "n_test_passed"] = (
                int(df.loc[distribution_name, ("chi_square", "rejected")] == False)
                + int(df.loc[distribution_name, ("kolmogorov_smirnov", "rejected")] == False)
                + int(df.loc[distribution_name, ("anderson_darling", "rejected")] == False)
            )
            df.loc[distribution_name, "n_test_nan"] = (
                int(numpy.isnan(df.loc[distribution_name, ("chi_square", "rejected")]))
                + int(numpy.isnan(df.loc[distribution_name, ("kolmogorov_smirnov", "rejected")]))
                + int(numpy.isnan(df.loc[distribution_name, ("anderson_darling", "rejected")]))
            )


alpha
0.013123482958133686
arcsine
0.07386683861892565
argus
inf
beta
nan
beta_prime
nan
beta_prime_4p


nan
bradford
0.03076632509143945
burr
0.015673304820764466
burr_4p
0.01760968871950788
cauchy
0.030557595166106163
chi_square
0.05009881073390405
chi_square_3p
0.011045469758323429
dagum
0.014075415919740562
dagum_4p
0.10604805146491905
erlang
0.013763953618952126
erlang_3p
0.013174833171342592
error_function
0.14695120186911556
exponential
0.11251808343118692
exponential_2p
0.5819143336249201
f
nan
fatigue_life
0.01201102268743812
folded_normal
0.014393107607925587
frechet
0.012298716496344493
f_4p
nan
gamma
0.013389762070880024
gamma_3p
0.013205962096901287
generalized_extreme_value
0.012518546873999052
generalized_gamma
inf
generalized_gamma_4p
nan
generalized_logistic
0.018825716335524635
generalized_normal
0.005042168111610973
generalized_pareto
0.03839755496649971
gibrat
0.025607012070345186
gumbel_left
0.033303336090728684
gumbel_right
0.019042275045497225
half_normal
0.04228298230623485
hyperbolic_secant
0.0339502776929013
inverse_gamma
inf
inverse_gamma_3p
inf
inverse_gaussian

In [89]:
df

chi_square                                    \
                           test_statistic critical_value   p_value rejected   
alpha                           75.702204      15.507313       0.0     True   
arcsine                        564.616046      16.918978       0.0     True   
argus                          255.070907      15.507313       0.0     True   
bradford                       187.785904      15.507313       0.0     True   
burr                            115.03298      15.507313       0.0     True   
burr_4p                         99.321784       14.06714       0.0     True   
cauchy                         282.112267      16.918978       0.0     True   
chi_square                     564.986492      18.307038       0.0     True   
chi_square_3p                   73.951048      15.507313       0.0     True   
dagum                          101.553039      15.507313       0.0     True   
erlang                          74.587895      16.918978       0.0     True   
erlang_3p                       71.661273      15.507313       0.0     True   
exponential                   4398.539692      18.307038       0.0     True   
exponential_2p              150181.054001      16.918978       0.0     True   
fatigue_life                    75.289975      15.507313       0.0     True   
folded_normal                   76.693685      16.918978       0.0     True   
frechet                       3043.733635      15.507313       0.0     True   
gamma                           72.334602      16.918978       0.0     True   
gamma_3p                        71.794277      15.507313       0.0     True   
generalized_extreme_value       68.101422      15.507313       0.0     True   
generalized_gamma               72.214116      15.507313       0.0     True   
generalized_logistic           112.643712      15.507313       0.0     True   
generalized_normal              43.832663      15.507313  0.000001     True   
generalized_pareto             242.526884      15.507313       0.0     True   
gibrat                         237.969371      16.918978       0.0     True   
gumbel_left                   3695.459525      16.918978       0.0     True   
gumbel_right                   129.275325      16.918978       0.0     True   
half_normal                    166.231921      16.918978       0.0     True   
hyperbolic_secant              191.596675      16.918978       0.0     True   
inverse_gamma                52862.801746      16.918978       0.0     True   
inverse_gamma_3p                73.167347      15.507313       0.0     True   
inverse_gaussian                72.088522      16.918978       0.0     True   
inverse_gaussian_3p             72.256917      15.507313       0.0     True   
johnson_sb                      30.295846       14.06714  0.000084     True   
laplace                        287.569481      16.918978       0.0     True   
levy                           684.093624      16.918978       0.0     True   
loggamma                        91.144724      15.507313       0.0     True   
logistic                       137.130275      16.918978       0.0     True   
loglogistic                 210804.560878      16.918978       0.0     True   
loglogistic_3p                 105.226165      15.507313       0.0     True   
lognormal                        72.53399      16.918978       0.0     True   
maxwell                         59.197711      16.918978       0.0     True   
moyal                           319.03432      16.918978       0.0     True   
nakagami                        72.288314      16.918978       0.0     True   
nc_f                            72.705969      15.507313       0.0     True   
normal                          76.693685      16.918978       0.0     True   
pert                            45.725901      15.507313       0.0     True   
rayleigh                         56.79707      16.918978       0.0     True   
reciprocal                     181.434058      16.918978       0.0     True   
ri

In [90]:
df.sort_values(by=["sse"])

chi_square                                    \
                           test_statistic critical_value   p_value rejected   
johnson_sb                      30.295846       14.06714  0.000084     True   
generalized_normal              43.832663      15.507313  0.000001     True   
triangular                      44.337029      15.507313       0.0     True   
pert                            45.725901      15.507313       0.0     True   
rayleigh                         56.79707      16.918978       0.0     True   
maxwell                         59.197711      16.918978       0.0     True   
weibull_3p                      55.241958      15.507313       0.0     True   
chi_square_3p                   73.951048      15.507313       0.0     True   
fatigue_life                    75.289975      15.507313       0.0     True   
frechet                       3043.733635      15.507313       0.0     True   
generalized_extreme_value       68.101422      15.507313       0.0     True   
alpha                           75.702204      15.507313       0.0     True   
inverse_gaussian                72.088522      16.918978       0.0     True   
erlang_3p                       71.661273      15.507313       0.0     True   
gamma_3p                        71.794277      15.507313       0.0     True   
lognormal                        72.53399      16.918978       0.0     True   
inverse_gaussian_3p             72.256917      15.507313       0.0     True   
nc_f                            72.705969      15.507313       0.0     True   
gamma                           72.334602      16.918978       0.0     True   
nakagami                        72.288314      16.918978       0.0     True   
erlang                          74.587895      16.918978       0.0     True   
dagum                          101.553039      15.507313       0.0     True   
rice                            76.650259      16.918978       0.0     True   
normal                          76.693685      16.918978       0.0     True   
folded_normal                   76.693685      16.918978       0.0     True   
burr                            115.03298      15.507313       0.0     True   
loglogistic_3p                 105.226165      15.507313       0.0     True   
loggamma                        91.144724      15.507313       0.0     True   
burr_4p                         99.321784       14.06714       0.0     True   
semicircular                   139.361303      16.918978       0.0     True   
generalized_logistic           112.643712      15.507313       0.0     True   
gumbel_right                   129.275325      16.918978       0.0     True   
weibull                        227.561511      16.918978       0.0     True   
logistic                       137.130275      16.918978       0.0     True   
gibrat                         237.969371      16.918978       0.0     True   
reciprocal                     181.434058      16.918978       0.0     True   
moyal                           319.03432      16.918978       0.0     True   
cauchy                         282.112267      16.918978       0.0     True   
bradford                       187.785904      15.507313       0.0     True   
gumbel_left                   3695.459525      16.918978       0.0     True   
uniform                        217.147541      16.918978       0.0     True   
hyperbolic_secant              191.596675      16.918978       0.0     True   
generalized_pareto             242.526884      15.507313       0.0     True   
half_normal                    166.231921      16.918978       0.0     True   
chi_square                     564.986492      18.307038       0.0     True   
levy                           684.093624      16.918978       0.0     True   
laplace                        287.569481      16.918978       0.0     True   
arcsine                        564.616046      16.918978       0.0     True   
exponential                   4398.539692      18.307038       0.0     True   
lo

In [91]:
dfx = df[(df[("chi_square", "rejected")] == False) | (df[("kolmogorov_smirnov", "rejected")] == False) | (df[("anderson_darling", "rejected")] == False)]
dfx = dfx.sort_values(by=["sse"])
dfx

Empty DataFrame
Columns: [(chi_square, test_statistic), (chi_square, critical_value), (chi_square, p_value), (chi_square, rejected), (kolmogorov_smirnov, test_statistic), (kolmogorov_smirnov, critical_value), (kolmogorov_smirnov, p_value), (kolmogorov_smirnov, rejected), (anderson_darling, test_statistic), (anderson_darling, critical_value), (anderson_darling, p_value), (anderson_darling, rejected), (sse, ), (parameters, ), (n_test_passed, ), (n_test_nan, )]
Index: []

In [92]:
dfx["parameters"] = dfx["parameters"].apply(eval)
dfx.columns = dfx.columns.map("_".join)
dfx


,chi_square_test_statistic,chi_square_critical_value,chi_square_p_value,chi_square_rejected,kolmogorov_smirnov_test_statistic,kolmogorov_smirnov_critical_value,kolmogorov_smirnov_p_value,kolmogorov_smirnov_rejected,anderson_darling_test_statistic,anderson_darling_critical_value,anderson_darling_p_value,anderson_darling_rejected,sse_,parameters_,n_test_passed_,n_test_nan_


In [93]:
data_dict = {}
for index, row in dfx.iterrows():
    data_dict[index] = {
        "chi_square": {
            "test_statistic": row["chi_square_test_statistic"],
            "critical_value": row["chi_square_critical_value"],
            "p_value": row["chi_square_p_value"],
            "rejected": row["chi_square_rejected"],
        },
        "kolmogorov_smirnov": {
            "test_statistic": row["kolmogorov_smirnov_test_statistic"],
            "critical_value": row["kolmogorov_smirnov_critical_value"],
            "p_value": row["kolmogorov_smirnov_p_value"],
            "rejected": row["kolmogorov_smirnov_rejected"],
        },
        "anderson_darling": {
            "test_statistic": row["anderson_darling_test_statistic"],
            "critical_value": row["anderson_darling_critical_value"],
            "p_value": row["anderson_darling_p_value"],
            "rejected": row["anderson_darling_rejected"],
        },
        "sse": row["sse_"],
        "parameters": row["parameters_"],
    }
data_dict

{}

In [94]:
dfx.to_excel("./test.xlsx")